In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
import math
import matplotlib.pyplot as plt

In [2]:
str_files = ''

#df = pd.read_csv(str_files + "ph.csv",encoding='utf-8',sep=';')
dateparse = lambda dates:pd.datetime.strptime(dates,'%d/%m/%Y')
df = pd.read_csv(str_files + 'ph.csv',encoding='utf-8',sep=';', parse_dates = ['data_coleta'],index_col=['data_coleta'],date_parser=dateparse)

In [3]:
z = df.loc[df['UGRHI'] == '10 - SOROCABA/MEDIO TIETE']

In [4]:
z = z.dropna(axis=1,how='any')

In [5]:
trainingd = z.resample('3M').mean()
trainingd
data = trainingd.iloc[:,1:2].values
train = trainingd.iloc[0:132,1:2].values  
test = trainingd.iloc[132:,1:2].values   
print('treino:',len(train))
print('teste:',len(test))

treino: 132
teste: 34


In [15]:
sc = MinMaxScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

In [16]:
def prepare_data(data, lags):
    X, y = [], []
    for row in range(len(data) - lags - 1):
        a = data[row:(row + lags), 0]
        X.append(a)
        y.append(data[row + lags, 0])
    return np.array(X), np.array(y)  

In [17]:
lags = 1
train_X, train_y = prepare_data(train, lags)
test_X, test_y = prepare_data(test, lags)

train_X = np.reshape(train_X, (train_X.shape[0],1,train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0],1,test_X.shape[1]))
print('Novo formato de treino:')
print(train_X.shape) 

Novo formato de treino:
(130, 1, 1)


In [18]:
model = Sequential() 

model.add(LSTM(units = 10, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1,patience=5)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 20s 151ms/sample - loss: 0.2727 - val_loss: 0.2799
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2619 - val_loss: 0.2690
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2511 - val_loss: 0.2580
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2397 - val_loss: 0.2469
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2294 - val_loss: 0.2357
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2180 - val_loss: 0.2244
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2087 - val_loss: 0.2132
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1968 - val_loss: 0.2015
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1857 - val_loss: 0.1896
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0461 - val_loss: 0.0539
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0451 - val_loss: 0.0539
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0410 - val_loss: 0.0539
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0370 - val_loss: 0.0538
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0400 - val_loss: 0.0540
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0417 - val_loss: 0.0542
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0408 - val_loss: 0.0542
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0439 - val_loss: 0.0543
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0433 - val_loss: 0.0544
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [19]:
train_score10 = model.evaluate(train_X, train_y, verbose=0)
print(train_score10)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score10, math.sqrt(train_score10)))
test_score10 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score10, math.sqrt(test_score10)))

0.03396206639277247
Pontuação de Treino: 0.03 MSE (0.18 RMSE)
Pontuação de teste: 0.06 MSE (0.24 RMSE)


In [21]:
model = Sequential() 

model.add(LSTM(units = 25, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='min',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 17s 128ms/sample - loss: 0.2726 - val_loss: 0.2779
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2592 - val_loss: 0.2640
Epoch 3/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.2460 - val_loss: 0.2494
Epoch 4/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.2306 - val_loss: 0.2339
Epoch 5/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.2146 - val_loss: 0.2172
Epoch 6/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.1996 - val_loss: 0.1995
Epoch 7/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.1838 - val_loss: 0.1807
Epoch 8/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.1617 - val_loss: 0.1614
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1427 - val_loss: 0.1417
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0383 - val_loss: 0.0599
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0389 - val_loss: 0.0603
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0349 - val_loss: 0.0600
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0360 - val_loss: 0.0600
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0376 - val_loss: 0.0602
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0380 - val_loss: 0.0606
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0344 - val_loss: 0.0612
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0397 - val_loss: 0.0603
Epoch 87/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0382 - val_loss: 0.0597
Epoch 88/100
130/130 [==============================] - 0s 2ms/sample - loss:

In [22]:
train_score25 = model.evaluate(train_X, train_y, verbose=0)
print(train_score25)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score25, math.sqrt(train_score25)))
test_score25 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score25, math.sqrt(test_score25)))

0.0313274949708452
Pontuação de Treino: 0.03 MSE (0.18 RMSE)
Pontuação de teste: 0.06 MSE (0.25 RMSE)


In [23]:
model = Sequential() 

model.add(LSTM(units = 50, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 17s 131ms/sample - loss: 0.2709 - val_loss: 0.2722
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2516 - val_loss: 0.2519
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2316 - val_loss: 0.2297
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2094 - val_loss: 0.2054
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1849 - val_loss: 0.1773
Epoch 6/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.1568 - val_loss: 0.1461
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1221 - val_loss: 0.1119
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0876 - val_loss: 0.0792
Epoch 9/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0607 - val_loss: 0.0567
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0312 - val_loss: 0.0636
Epoch 80/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0327 - val_loss: 0.0628
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0354 - val_loss: 0.0630
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0334 - val_loss: 0.0639
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0328 - val_loss: 0.0643
Epoch 84/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0360 - val_loss: 0.0653
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0345 - val_loss: 0.0660
Epoch 86/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0343 - val_loss: 0.0659
Epoch 87/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0333 - val_loss: 0.0656
Epoch 88/100
130/130 [==============================] - 0s 2ms/sample - loss:

In [25]:
train_score50 = model.evaluate(train_X, train_y, verbose=0)
print(train_score50)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score50, math.sqrt(train_score50)))
test_score50 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score50, math.sqrt(test_score50)))

0.030898405730616873
Pontuação de Treino: 0.03 MSE (0.18 RMSE)
Pontuação de teste: 0.06 MSE (0.25 RMSE)
